In [1]:
import json
from abc import ABC, abstractmethod
from datetime import datetime, timedelta


In [2]:
class LibraryItem(ABC):
    def __init__(self, title, author, item_id):
        self.title = title
        self.author = author
        self.item_id = item_id
        self.__is_borrowed = False

    @abstractmethod
    def item_type(self):
        pass

    def borrow(self):
        if self.__is_borrowed:
            raise Exception(f"{self.title} is already borrowed.")
        self.__is_borrowed = True

    def return_item(self):
        if not self.__is_borrowed:
            raise Exception(f"{self.title} was not borrowed.")
        self.__is_borrowed = False

    def is_borrowed(self):
        return self.__is_borrowed


In [3]:
class Book(LibraryItem):
    def item_type(self):
        return "Book"

class Magazine(LibraryItem):
    def item_type(self):
        return "Magazine"


In [4]:
class Member:
    member_count = 0  # class attribute

    def __init__(self, name):
        self.name = name
        self.member_id = Member.member_count + 1
        Member.member_count += 1
        self.borrowed_items = {}  # {item_id: due_date}

    def borrow_item(self, item, days=14):
        if item.is_borrowed():
            raise Exception(f"{item.title} is already borrowed")
        item.borrow()
        due_date = datetime.now() + timedelta(days=days)
        self.borrowed_items[item.item_id] = due_date
        print(f"{self.name} borrowed {item.title}, due on {due_date.date()}")

    def return_item(self, item):
        if item.item_id not in self.borrowed_items:
            raise Exception(f"{self.name} did not borrow {item.title}")
        item.return_item()
        del self.borrowed_items[item.item_id]
        print(f"{self.name} returned {item.title}")


In [5]:
class LibrarySystem:
    def __init__(self):
        self.items = {}   # {item_id: LibraryItem}
        self.members = {} # {member_id: Member}

    def add_item(self, item):
        self.items[item.item_id] = item
        print(f"{item.item_type()} '{item.title}' added to library")

    def add_member(self, member):
        self.members[member.member_id] = member
        print(f"Member '{member.name}' added")

    def borrow_item(self, member_id, item_id):
        try:
            member = self.members[member_id]
            item = self.items[item_id]
            member.borrow_item(item)
        except KeyError:
            print("Invalid member or item ID")
        except Exception as e:
            print("Error:", e)

    def return_item(self, member_id, item_id):
        try:
            member = self.members[member_id]
            item = self.items[item_id]
            member.return_item(item)
        except KeyError:
            print("Invalid member or item ID")
        except Exception as e:
            print("Error:", e)

    def save_data(self, filename):
        data = {
            "items": [{ "id": i.item_id, "title": i.title, "author": i.author,
                        "type": i.item_type(), "borrowed": i.is_borrowed() } 
                        for i in self.items.values()],
            "members": [{ "id": m.member_id, "name": m.name, 
                          "borrowed_items": {k:str(v) for k,v in m.borrowed_items.items()}} 
                          for m in self.members.values()]
        }
        with open(filename, "w") as f:
            json.dump(data, f, indent=4)
        print("Library data saved.")

    def load_data(self, filename):
        try:
            with open(filename, "r") as f:
                data = json.load(f)
            # Load items
            for i in data["items"]:
                if i["type"] == "Book":
                    item = Book(i["title"], i["author"], i["id"])
                else:
                    item = Magazine(i["title"], i["author"], i["id"])
                if i["borrowed"]:
                    item.borrow()
                self.items[item.item_id] = item
            # Load members
            for m in data["members"]:
                member = Member(m["name"])
                member.member_id = m["id"]
                # Convert due_date strings back to datetime
                member.borrowed_items = {int(k):datetime.fromisoformat(v) 
                                         for k,v in m["borrowed_items"].items()}
                self.members[member.member_id] = member
            print("Library data loaded.")
        except FileNotFoundError:
            print("File not found.")


In [6]:
# Create library system
library = LibrarySystem()

# Add items
book1 = Book("Python Programming", "John Doe", 1)
book2 = Book("Machine Learning", "Jane Smith", 2)
mag1 = Magazine("AI Monthly", "Tech Media", 3)
library.add_item(book1)
library.add_item(book2)
library.add_item(mag1)

# Add members
member1 = Member("Alice")
member2 = Member("Bob")
library.add_member(member1)
library.add_member(member2)

# Borrow and return
library.borrow_item(1, 1)  # Alice borrows Python Programming
library.borrow_item(2, 1)  # Bob tries to borrow already borrowed book
library.return_item(1, 1)  # Alice returns book
library.borrow_item(2, 1)  # Now Bob borrows it

# Save and load
library.save_data("library.json")
new_library = LibrarySystem()
new_library.load_data("library.json")


Book 'Python Programming' added to library
Book 'Machine Learning' added to library
Magazine 'AI Monthly' added to library
Member 'Alice' added
Member 'Bob' added
Alice borrowed Python Programming, due on 2025-10-30
Error: Python Programming is already borrowed
Alice returned Python Programming
Bob borrowed Python Programming, due on 2025-10-30
Library data saved.
Library data loaded.
